# Git & GitHub Learning Lab (Jupyter Notebook)

**Last updated:** 2026-02-07

This notebook is a hands-on lab to learn Git and GitHub.

**How to use it**
- Read each section, then run the commands in the code cells.
- Most code cells use shell commands (prefixed with `!`) so they work in Jupyter.
- If you are in GitHub Codespaces, you already have Git installed.

> Tip: If a command fails, read the error message carefully—Git errors are usually very informative.


## Learning goals
By the end you will be able to:
- Explain what a repository, commit, branch, and remote are
- Create commits with clear messages
- Use branches and pull requests (PRs)
- Sync changes with GitHub (push/pull)
- Resolve a simple merge conflict
- Use `.gitignore` and review history


## 0) Quick check: where am I and do I have Git?
Run the cell below to confirm your environment.

In [ ]:
# Environment check
!pwd
!ls -la
!git --version


## 1) Core ideas (quick mental model)
- **Working directory**: your files
- **Staging area (index)**: what will go into the next commit
- **Repository (.git)**: Git’s database of commits

A typical cycle:
1. Edit files
2. `git add` (stage)
3. `git commit` (snapshot)
4. `git push` (share)


## 2) Create a practice repo
We’ll make a new folder and initialize a Git repository.

In [ ]:
# Create a practice folder (safe & self-contained)
!mkdir -p git_github_lab
%cd git_github_lab

# Initialize a Git repository
!git init

# See what's in the folder
!ls -la


### Configure your name/email (one-time per machine)
If Git complains about identity, set it. Use the email you use on GitHub.

In [ ]:
# OPTIONAL (run if needed): set your name and email
# Replace with your info
# !git config --global user.name "Your Name"
# !git config --global user.email "you@example.com"

# Show current config
!git config --global --get user.name
!git config --global --get user.email


## 3) Your first commit
We’ll add a README and commit it.

In [ ]:
# Create a README
readme = """# Git & GitHub Lab

This is a practice repo for learning Git.
""
with open('README.md','w') as f:
    f.write(readme)

!git status


In [ ]:
# Stage and commit
!git add README.md
!git status

!git commit -m "Add initial README"
!git log --oneline --decorate --graph -5


## 4) Make changes and understand the staging area
Edit the README, view diffs, and commit again.

In [ ]:
# Edit README (append a line)
with open('README.md','a') as f:
    f.write("\n## Notes\n- Practicing commits and branches.\n")

!git status
!git diff  # shows unstaged changes


In [ ]:
# Stage and view staged diff
!git add README.md
!git diff --staged  # shows what will be committed

!git commit -m "Add notes section"
!git log --oneline --decorate --graph -5


## 5) Branching basics
Branches let you work on changes without touching the main line.

We'll create a feature branch, make a change, and merge it back.

In [ ]:
# Create and switch to a new branch
!git switch -c feature/add-file

# Create a new file
with open('hello.txt','w') as f:
    f.write("Hello from a feature branch!\n")

!git status
!git add hello.txt
!git commit -m "Add hello file"

!git log --oneline --decorate --graph -8


In [ ]:
# Switch back to main (sometimes it's called master)
# We'll detect the default branch name
import subprocess, shlex, os, re, textwrap, json
branches = subprocess.check_output(["bash","-lc","git branch --format='%(refname:short)'"]).decode().splitlines()
default = "main" if "main" in branches else ("master" if "master" in branches else branches[0])
print("Default branch:", default)

!git switch {default}
!git merge feature/add-file

!git log --oneline --decorate --graph -10


## 6) Remotes and GitHub
A **remote** is a nickname for another copy of your repo, often on GitHub.

### Option A: If this repo already lives on GitHub (common in class)
Run `git remote -v` to see it.

### Option B: Create a new repo on GitHub and connect it
1. Create an empty repo on GitHub (no README).
2. Copy the repo URL (HTTPS).
3. Add it as `origin`, then push.

> In Codespaces you can authenticate with GitHub CLI (`gh`) if available.


In [ ]:
# Check existing remotes
!git remote -v


In [ ]:
# OPTION B (edit before running):
# 1) Replace the URL with your GitHub repo URL, e.g. https://github.com/<user>/<repo>.git
# 2) Then push your default branch.
# 
# repo_url = "PASTE_YOUR_GITHUB_REPO_URL_HERE"
# !git remote add origin {repo_url}
# !git push -u origin {default}


## 7) Pull updates (and why you should do it often)
If other changes land on GitHub, you bring them down with `git pull`.

- `git pull` = `git fetch` + `git merge`
- For a cleaner history, some teams prefer `git pull --rebase`


In [ ]:
# If you have a remote and want to sync:
# !git pull

# Show remote branches (if remote exists)
!git branch -a


## 8) Merge conflicts (safe practice)
Conflicts happen when Git can't automatically combine changes.

We'll simulate one locally with two branches editing the same line.


In [ ]:
# Create a file to conflict on
with open('conflict.txt','w') as f:
    f.write("Line 1: original\n")

!git add conflict.txt
!git commit -m "Add conflict practice file"


In [ ]:
# Branch A changes the same line
!git switch -c branch-A
with open('conflict.txt','w') as f:
    f.write("Line 1: change from branch A\n")
!git add conflict.txt
!git commit -m "Branch A edits line 1"


In [ ]:
# Switch back to default and create Branch B
!git switch {default}
!git switch -c branch-B
with open('conflict.txt','w') as f:
    f.write("Line 1: change from branch B\n")
!git add conflict.txt
!git commit -m "Branch B edits line 1"


In [ ]:
# Merge Branch A into Branch B to trigger conflict
# (This should create a conflict on conflict.txt)
!git merge branch-A


### Resolve the conflict
1. Open `conflict.txt` and choose the final content.
2. Stage the file
3. Commit the merge

Run the helper cell below to show the conflict markers and then finish the steps.

In [ ]:
# Show the conflict file content (with conflict markers)
print(open('conflict.txt').read())

# After you fix it, run:
# !git add conflict.txt
# !git commit -m "Resolve merge conflict in conflict.txt"


## 9) Undoing mistakes safely
Common tools:
- `git restore <file>`: discard unstaged changes in a file
- `git restore --staged <file>`: unstage a file
- `git revert <commit>`: create a new commit that undoes a previous commit (safe for shared history)
- `git reset` (advanced): rewrites history—use carefully


In [ ]:
# Practice: create a temporary change, then restore it
with open('temp.txt','w') as f:
    f.write("temporary\n")
!git status

# Discard by removing the file (not a Git command)
import os
os.remove('temp.txt')
!git status


## 10) .gitignore
Use `.gitignore` to keep files out of version control (like secrets, caches, datasets).

In [ ]:
# Create a .gitignore
gitignore = """# Python
__pycache__/
*.pyc

# Jupyter checkpoints
.ipynb_checkpoints/

# OS
.DS_Store

# Secrets (never commit!)
.env
""
with open('.gitignore','w') as f:
    f.write(gitignore)

!git add .gitignore
!git commit -m "Add .gitignore"
!git status


## 11) Good commit messages (quick guide)
Aim for: **imperative mood** + **what/why**
- ✅ `Add data dictionary for cardio dataset`
- ✅ `Fix off-by-one error in risk score loop`
- ❌ `stuff`
- ❌ `updates`


## 12) GitHub workflow: Pull Requests (PRs)
Typical team workflow:
1. Create a branch (`feature/...`)
2. Make commits
3. Push branch to GitHub (`git push -u origin feature/...`)
4. Open a PR
5. Review + checks
6. Merge into main

If you want practice, create a new branch, change the README, push, and open a PR on GitHub.


In [ ]:
# PR practice (edit & run as needed)
# !git switch -c feature/pr-practice
# (edit files, commit)
# !git push -u origin feature/pr-practice


## 13) Reference cheat sheet
**Daily basics**
- `git status`
- `git add <file>`
- `git commit -m "message"`
- `git pull`
- `git push`

**Branching**
- `git switch -c <branch>`
- `git switch <branch>`
- `git merge <branch>`

**History**
- `git log --oneline --graph --decorate`
- `git diff`

**Remotes**
- `git remote -v`
- `git push -u origin <branch>`


## 14) Mini-checkpoint (self-assessment)
Try answering without looking:
1. What is the difference between working directory and staging area?
2. What does `git commit` create?
3. Why use branches?
4. What is a remote?
5. What is a PR used for?

If you can answer these clearly, you’re in good shape.
